In [ ]:
import transformers
import torch
import pandas as pd
import numpy as np
import re
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, BitsAndBytesConfig, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm.notebook import tqdm
pd.options.display.max_colwidth = 1000

In [ ]:
merged_df = pd.read_csv('merged_df.csv') ## AI-Hub에서 가장 처음 가져온 원데이터
jeju = pd.read_csv('제주발화.csv') ## 다른 AI-Hub에서 가져온 방언 발화인데 제주만 뽑은 데이터
life = pd.read_excel('생활제주어.xlsx') ## 제주어사전에서 가져온 생활 제주어 데이터
je_ko = pd.read_csv('je_ko.csv') ## 아래아 처리한 카카오 jit 데이터

<ipython-input-3-51711a58ce29>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('merged_df.csv') ## AI-Hub에서 가장 처음 가져온 원데이터


## merged_df부터 전처리
- 더 완벽한 데이터 질을 위해서!!

In [ ]:
## 제주어 - 표준어 서로의 답이 있는 경우만 남기기
merged_df = merged_df.loc[merged_df.isDialect == True]

## 불용어제거
merged_df['standard_form'] = merged_df['standard_form'].apply(lambda x : x.replace('#', ''))
merged_df['dialect_form'] = merged_df['dialect_form'].apply(lambda x : x.replace('#', ''))
merged_df['standard_form'] = merged_df['standard_form'].apply(lambda x : x.replace('@', ''))
merged_df['dialect_form'] = merged_df['dialect_form'].apply(lambda x : x.replace('@', ''))
merged_df['standard_form'] = merged_df['standard_form'].apply(lambda x : x.replace('*', ''))
merged_df['dialect_form'] = merged_df['dialect_form'].apply(lambda x : x.replace('*', ''))
merged_df['standard_form'] = merged_df['standard_form'].apply(lambda x : x.replace('/', ''))
merged_df['dialect_form'] = merged_df['dialect_form'].apply(lambda x : x.replace('/', ''))
merged_df['standard_form'] = merged_df['standard_form'].apply(lambda x: re.sub(r'&[^&]*&', '', re.sub(r'\{[^}]*\}', '', re.sub(r'-[^-]*-', '', x)))) ## &name1&, {laughing}, -그- <-- 이런거 제거
merged_df['dialect_form'] = merged_df['dialect_form'].apply(lambda x: re.sub(r'&[^&]*&', '', re.sub(r'\{[^}]*\}', '', re.sub(r'-[^-]*-', '', x)))) ## &name1&, {laughing}, -그- <-- 이런거 제거

## 이중괄호 처리
def process_double_parentheses(text):
    ## "x", "xx", "xxx"를 포함하거나, 아무 것도 없는 이중 괄호 제거
    text = re.sub(r'\(\((x{1,3})?\)\)', '', text)
    ## 일반적인 이중 괄호 내 텍스트 추출
    return re.sub(r'\(\(([^)]+)\)\)', r'\1', text)
merged_df['standard_form'] = merged_df['standard_form'].apply(process_double_parentheses)
merged_df['dialect_form'] = merged_df['dialect_form'].apply(process_double_parentheses)

## "(text1)/(text2)" 패턴을 찾아서 첫 번째 텍스트만 추출하는 함수 및 적용
def simplify_dialect_form(text):
    return re.sub(r'\(([^)]+)\)\(([^)]+)\)', r'\1', text)
merged_df['dialect_form'] = merged_df['dialect_form'].apply(simplify_dialect_form)

## "(text1)/(text2)" 패턴을 찾아서 두 번째 텍스트만 추출하는 함수 및 적용
def simplify_standard_form(text):
    return re.sub(r'\(([^)]+)\)\(([^)]+)\)', r'\1', text)
merged_df['standard_form'] = merged_df['standard_form'].apply(simplify_standard_form)

## 전처리 후 ()가 지저분하게 남아 있는 데이터 제거
merged_df = merged_df[~merged_df['standard_form'].str.contains('\(', regex = True)]

## 제주어, 표준어가 같은 경우 제거
merged_df = merged_df.loc[merged_df.standard_form != merged_df.dialect_form]

## 출생이 제주인 사람만 남기기
merged_df = merged_df.loc[merged_df.birthplace == '제주']

In [ ]:
## (text1)/(text2) 제대로 번역이 되고 있는지 확인하는 함수
def extract_patterns(text):
    pattern = r'\(.*?\/.*?\)'
    matches = re.findall(pattern, text)
    return ', '.join(matches)

# 적용할 열이 'form'이라 가정하고 함수 적용
merged_df['extracted'] = merged_df['form'].apply(extract_patterns)

In [ ]:
## 사투리가 아닌데 나뉜 경우 제거
merged_df = merged_df.loc[merged_df.extracted != '(같애)/(같아)']
merged_df = merged_df.loc[merged_df.extracted != '(같애.)/(같아.)']
merged_df = merged_df.loc[merged_df.extracted != '(거)/(그거)']
merged_df = merged_df.loc[merged_df.extracted != '(거)/(거야)']
merged_df = merged_df.loc[merged_df.extracted != '(거?)/(거야?)']
merged_df = merged_df.loc[merged_df.extracted != '(거.)/(거야.)']
merged_df = merged_df.loc[merged_df.extracted != '(건)/(그건)']
merged_df = merged_df.loc[merged_df.extracted != '(것도)/(그것도)']
merged_df = merged_df.loc[merged_df.extracted != '(괜춘)/(괜찮아)']
merged_df = merged_df.loc[merged_df.extracted != '(그래가지고)/(그래서)']
merged_df = merged_df.loc[merged_df.extracted != '(걍)/(그냥)']
merged_df = merged_df.loc[merged_df.extracted != '(뭐?)/(뭐야?)']
merged_df = merged_df.loc[merged_df.extracted != '(뭔)/(무슨)']
merged_df = merged_df.loc[merged_df.extracted != '(싶)/(싶어)']
merged_df = merged_df.loc[merged_df.extracted != '(싶.)/(싶어.)']
merged_df = merged_df.loc[merged_df.extracted != '(아니)/(아니야)']
merged_df = merged_df.loc[merged_df.extracted != '(아니?)/(아니야?)']
merged_df = merged_df.loc[merged_df.extracted != '(않애?)/(않아?)']
merged_df = merged_df.loc[merged_df.extracted != '(해가지고)/(해서)']

## (근데)/(그런데) <-- 이 경우에는 근데를 겐디로 변경
merged_df.loc[merged_df.extracted == '(근데)/(그런데)', 'dialect_form'] = merged_df.loc[merged_df.extracted == '(근데)/(그런데)', 'dialect_form'].apply(lambda x: x.replace('근데', '겐디'))

## (그니까)/(그러니까) <-- 이 경우에는 그니까를 게난으로 변경
merged_df.loc[merged_df.extracted == '(그니까)/(그러니까)', 'dialect_form'] = merged_df.loc[merged_df.extracted == '(그니까)/(그러니까)', 'dialect_form'].apply(lambda x: x.replace('그니까', '게난'))

## (그니까)/(그러니까) <-- 이 경우에는 그니까를 게난으로 변경
merged_df.loc[merged_df.extracted == '(진짜이)/(진짜)', 'dialect_form'] = merged_df.loc[merged_df.extracted == '(진짜이)/(진짜)', 'dialect_form'].apply(lambda x: x.replace('진짜이', '진짜'))

## (닮)/(같아) <-- 이 경우에는 닮을 같아로 변경, 띄어쓰기로 구분해서 변경
merged_df.loc[merged_df.extracted == '(닮)/(같아)', 'dialect_form'] = merged_df.loc[merged_df.extracted == '(닮)/(같아)', 'dialect_form'].apply(lambda x: ' '.join(word.replace('닮', '같아') if word == '닮' else word for word in x.split()))
merged_df.loc[merged_df.extracted == '(닮.)/(같아.)', 'dialect_form'] = merged_df.loc[merged_df.extracted == '(닮.)/(같아.)', 'dialect_form'].apply(lambda x: ' '.join(word.replace('닮.', '같아.') if word == '닮.' else word for word in x.split()))

## 발음 교정
## 갓, 것, 낫, 냇, 놧, 밧, 봣, 앗, 엇, 왓, 잇, 핫, 헛
## 제주어 받침으로는 쌍시옷이 올 수 없다
merged_df.loc[merged_df.dialect_form.str.contains('갔'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('갔'), 'dialect_form'].apply(lambda x: x.replace('갔', '갓'))
merged_df.loc[merged_df.dialect_form.str.contains('겄'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('겄'), 'dialect_form'].apply(lambda x: x.replace('겄', '것'))
merged_df.loc[merged_df.dialect_form.str.contains('났'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('났'), 'dialect_form'].apply(lambda x: x.replace('났', '낫'))
merged_df.loc[merged_df.dialect_form.str.contains('놨'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('놨'), 'dialect_form'].apply(lambda x: x.replace('놨', '놧'))
merged_df.loc[merged_df.dialect_form.str.contains('밨'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('밨'), 'dialect_form'].apply(lambda x: x.replace('밨', '밧'))
merged_df.loc[merged_df.dialect_form.str.contains('봤'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('봤'), 'dialect_form'].apply(lambda x: x.replace('봤', '봣'))
merged_df.loc[merged_df.dialect_form.str.contains('았'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('았'), 'dialect_form'].apply(lambda x: x.replace('았', '앗'))
merged_df.loc[merged_df.dialect_form.str.contains('었'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('었'), 'dialect_form'].apply(lambda x: x.replace('었', '엇'))
merged_df.loc[merged_df.dialect_form.str.contains('왔'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('왔'), 'dialect_form'].apply(lambda x: x.replace('왔', '왓'))
merged_df.loc[merged_df.dialect_form.str.contains('있'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('있'), 'dialect_form'].apply(lambda x: x.replace('있', '잇'))
merged_df.loc[merged_df.dialect_form.str.contains('핬'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('핬'), 'dialect_form'].apply(lambda x: x.replace('핬', '핫'))
merged_df.loc[merged_df.dialect_form.str.contains('헜'), 'dialect_form'] = merged_df.loc[merged_df.dialect_form.str.contains('헜'), 'dialect_form'].apply(lambda x: x.replace('헜', '헛'))

## 제주어, 표준어가 같은 경우 한 번 더 제거
merged_df = merged_df.loc[merged_df.standard_form != merged_df.dialect_form]

In [ ]:
## 중복 행 제거
merged_df = merged_df.drop_duplicates(subset='standard_form', keep='first')
merged_df = merged_df.drop_duplicates(subset='dialect_form', keep='first')

In [ ]:
## 발화 길이 조정
merged_df['standard_len'] = merged_df.standard_form.apply(lambda x : len(x))
merged_df['dialect_len'] = merged_df.dialect_form.apply(lambda x : len(x))
merged_df = merged_df.loc[(merged_df.standard_len >= 4) & (merged_df.standard_len < 64)]
merged_df = merged_df.loc[(merged_df.dialect_len >= 4) & (merged_df.dialect_len < 64)]

## 두 column의 len 차이가 8 보다 작은 데이터만 남기기 --> 8 이상인 경우에 옳은 데이터도 있지만 ()나 이상한 단어가 많이 껴 있는 경우가 많았음
merged_df = merged_df[abs(merged_df.standard_len - merged_df.dialect_len) < 8]

In [ ]:
## 번역 정확도를 올리기 위해 데이터 변형
## 1) 인사말
merged_df.loc[merged_df.standard_form.str.contains('안녕하세요'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('안녕하세요'), 'dialect_form'].apply(lambda x: x.replace('안녕하세요', '안녕하수꽈'))

## 2) 동물
merged_df.loc[merged_df.standard_form.str.contains('가리비'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('가리비'), 'dialect_form'].apply(lambda x: x.replace('가리비', '밥조개'))
merged_df.loc[merged_df.standard_form.str.contains('가물치'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('가물치'), 'dialect_form'].apply(lambda x: x.replace('가물치', '가마치'))
merged_df.loc[merged_df.standard_form.str.contains('가자미'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('가자미'), 'dialect_form'].apply(lambda x: x.replace('가자미', '가제미'))
merged_df.loc[merged_df.standard_form.str.contains('강아지'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('강아지'), 'dialect_form'].apply(lambda x: x.replace('강아지', '강생이'))
merged_df.loc[merged_df.standard_form.str.contains('개구리'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('개구리'), 'dialect_form'].apply(lambda x: x.replace('개구리', '골개비'))
merged_df.loc[merged_df.standard_form.str.contains('고등어'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('고등어'), 'dialect_form'].apply(lambda x: x.replace('고등어', '고등에'))
merged_df.loc[merged_df.standard_form.str.contains('고양이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('고양이'), 'dialect_form'].apply(lambda x: x.replace('고양이', '고냉이'))
merged_df.loc[merged_df.standard_form.str.contains('꽃게'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('꽃게'), 'dialect_form'].apply(lambda x: x.replace('꽃게', '꺽겡이'))
merged_df.loc[merged_df.standard_form.str.contains('꾀꼬리'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('꾀꼬리'), 'dialect_form'].apply(lambda x: x.replace('꾀꼬리', '꿰꼬리'))
merged_df.loc[merged_df.standard_form.str.contains('넙치'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('넙치'), 'dialect_form'].apply(lambda x: x.replace('넙치', '광어'))
merged_df.loc[merged_df.standard_form.str.contains('노루'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('노루'), 'dialect_form'].apply(lambda x: x.replace('노루', '노리'))
merged_df.loc[merged_df.standard_form.str.contains('닭'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('닭'), 'dialect_form'].apply(lambda x: x.replace('닭', '독'))
merged_df.loc[merged_df.standard_form.str.contains('달걀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('달걀'), 'dialect_form'].apply(lambda x: x.replace('달걀', '독새기'))
merged_df.loc[merged_df.standard_form.str.contains('달팽이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('달팽이'), 'dialect_form'].apply(lambda x: x.replace('달팽이', '돌뱅이'))
merged_df.loc[merged_df.standard_form.str.contains('도마뱀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('도마뱀'), 'dialect_form'].apply(lambda x: x.replace('도마뱀', '독다귀'))
merged_df.loc[merged_df.standard_form.str.contains('돼지'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('돼지'), 'dialect_form'].apply(lambda x: x.replace('돼지', '도새기'))
merged_df.loc[merged_df.standard_form.str.contains('딱따구리'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('딱따구리'), 'dialect_form'].apply(lambda x: x.replace('딱따구리', '남도래기'))
merged_df.loc[merged_df.standard_form.str.contains('망아지'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('망아지'), 'dialect_form'].apply(lambda x: x.replace('망아지', '몽생이'))
merged_df.loc[merged_df.standard_form.str.contains('매미'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('매미'), 'dialect_form'].apply(lambda x: x.replace('매미', '재열'))
merged_df.loc[merged_df.standard_form.str.contains('멧돼지'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멧돼지'), 'dialect_form'].apply(lambda x: x.replace('멧돼지', '산톳'))
merged_df.loc[merged_df.standard_form.str.contains('문어'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('문어'), 'dialect_form'].apply(lambda x: x.replace('문어', '물꾸럭'))
merged_df.loc[merged_df.standard_form.str.contains('멸치'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멸치'), 'dialect_form'].apply(lambda x: x.replace('멸치', '멜'))
merged_df.loc[merged_df.standard_form.str.contains('민달팽이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('민달팽이'), 'dialect_form'].apply(lambda x: x.replace('민달팽이', '옷벗은돌뱅이'))
merged_df.loc[merged_df.standard_form.str.contains('박쥐'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('박쥐'), 'dialect_form'].apply(lambda x: x.replace('박쥐', '다람쥐'))
merged_df.loc[merged_df.standard_form.str.contains('반딧불이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('반딧불이'), 'dialect_form'].apply(lambda x: x.replace('반딧불이', '불한지'))
merged_df.loc[merged_df.standard_form.str.contains('벌레'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('벌레'), 'dialect_form'].apply(lambda x: x.replace('벌레', '버렝이'))
merged_df.loc[merged_df.standard_form.str.contains('보리새우'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('보리새우'), 'dialect_form'].apply(lambda x: x.replace('보리새우', '돌새위'))
merged_df.loc[merged_df.standard_form.str.contains('복어'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('복어'), 'dialect_form'].apply(lambda x: x.replace('복어', '복쟁이'))
merged_df.loc[merged_df.standard_form.str.contains('뱀장어'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('뱀장어'), 'dialect_form'].apply(lambda x: x.replace('뱀장어', '민물장의'))
merged_df.loc[merged_df.standard_form.str.contains('붕장어'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('붕장어'), 'dialect_form'].apply(lambda x: x.replace('붕장어', '붕장의'))
merged_df.loc[merged_df.standard_form.str.contains('벼룩'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('벼룩'), 'dialect_form'].apply(lambda x: x.replace('벼룩', '베록'))
merged_df.loc[merged_df.standard_form.str.contains('병아리'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('병아리'), 'dialect_form'].apply(lambda x: x.replace('병아리', '빙애기'))
merged_df.loc[merged_df.standard_form.str.contains('소라'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('소라'), 'dialect_form'].apply(lambda x: x.replace('소라', '구젱기'))
merged_df.loc[merged_df.standard_form.str.contains('송아지'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('송아지'), 'dialect_form'].apply(lambda x: x.replace('송아지', '송애기'))
merged_df.loc[merged_df.standard_form.str.contains('얼룩소'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('얼룩소'), 'dialect_form'].apply(lambda x: x.replace('얼룩소', '어럭쉐'))
merged_df.loc[merged_df.standard_form.str.contains('얼룩소'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('얼룩소'), 'dialect_form'].apply(lambda x: x.replace('얼룩쉐', '어럭쉐'))
merged_df.loc[merged_df.standard_form.str.contains('오소리'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('오소리'), 'dialect_form'].apply(lambda x: x.replace('오소리', '지다리'))
merged_df.loc[merged_df.standard_form.str.contains('올빼미'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('올빼미'), 'dialect_form'].apply(lambda x: x.replace('올빼미', '옥빰'))
merged_df.loc[merged_df.standard_form.str.contains('올챙이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('올챙이'), 'dialect_form'].apply(lambda x: x.replace('올챙이', '멘주기'))
merged_df.loc[merged_df.standard_form.str.contains('왕거미'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('왕거미'), 'dialect_form'].apply(lambda x: x.replace('왕거미', '돗거미'))
merged_df.loc[merged_df.standard_form.str.contains('왕지네'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('왕지네'), 'dialect_form'].apply(lambda x: x.replace('왕지네', '황지넹이'))
merged_df.loc[merged_df.standard_form.str.contains('여우'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('여우'), 'dialect_form'].apply(lambda x: x.replace('여우', '여히'))
merged_df.loc[merged_df.standard_form.str.contains('염소'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('염소'), 'dialect_form'].apply(lambda x: x.replace('염소', '염쉐'))
merged_df.loc[merged_df.standard_form.str.contains('장어'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('장어'), 'dialect_form'].apply(lambda x: x.replace('장어', '장의'))
merged_df.loc[merged_df.standard_form.str.contains('전갱이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('전갱이'), 'dialect_form'].apply(lambda x: x.replace('전갱이', '각제기'))
merged_df.loc[merged_df.standard_form.str.contains('쥐치'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('쥐치'), 'dialect_form'].apply(lambda x: x.replace('쥐치', '겍주리'))
merged_df.loc[merged_df.standard_form.str.contains('지렁이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('지렁이'), 'dialect_form'].apply(lambda x: x.replace('지렁이', '지렝이'))
merged_df.loc[merged_df.standard_form.str.contains('지네'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('지네'), 'dialect_form'].apply(lambda x: x.replace('지네', '지넹이'))
merged_df.loc[merged_df.standard_form.str.contains('토끼'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('토끼'), 'dialect_form'].apply(lambda x: x.replace('토끼', '퉤끼'))

## 3) 일상단어
merged_df.loc[merged_df.standard_form.str.contains('꽁무니'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('꽁무니'), 'dialect_form'].apply(lambda x: x.replace('꽁무니', '미주'))
merged_df.loc[merged_df.standard_form.str.contains('눈동자'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('눈동자'), 'dialect_form'].apply(lambda x: x.replace('눈동자', '눈방울'))
merged_df.loc[merged_df.standard_form.str.contains('두부'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('두부'), 'dialect_form'].apply(lambda x: x.replace('두부', '둠비'))
merged_df.loc[merged_df.standard_form.str.contains('멱살'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멱살'), 'dialect_form'].apply(lambda x: x.replace('멱살', '모감지'))
merged_df.loc[merged_df.standard_form.str.contains('반찬'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('반찬'), 'dialect_form'].apply(lambda x: x.replace('반찬', '촐래'))
merged_df.loc[merged_df.standard_form.str.contains('발톱'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('발톱'), 'dialect_form'].apply(lambda x: x.replace('발톱', '발콥'))
merged_df.loc[merged_df.standard_form.str.contains('밥주걱'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('밥주걱'), 'dialect_form'].apply(lambda x: x.replace('밥주걱', '누금'))
merged_df.loc[merged_df.standard_form.str.contains('벌초'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('벌초'), 'dialect_form'].apply(lambda x: x.replace('벌초', '소본'))
merged_df.loc[merged_df.standard_form.str.contains('복숭아'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('복숭아'), 'dialect_form'].apply(lambda x: x.replace('복숭아', '복숭개'))
merged_df.loc[merged_df.standard_form.str.contains('생일'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('생일'), 'dialect_form'].apply(lambda x: x.replace('생일', '셍일'))
merged_df.loc[merged_df.standard_form.str.contains('소금'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('소금'), 'dialect_form'].apply(lambda x: x.replace('소금', '소곰'))
merged_df.loc[merged_df.standard_form.str.contains('소주'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('소주'), 'dialect_form'].apply(lambda x: x.replace('소주', '소쥐'))
merged_df.loc[merged_df.standard_form.str.contains('손톱'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('손톱'), 'dialect_form'].apply(lambda x: x.replace('손톱', '손콥'))
merged_df.loc[merged_df.standard_form.str.contains('시루떡'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('시루떡'), 'dialect_form'].apply(lambda x: x.replace('시루떡', '시리떡'))
merged_df.loc[merged_df.standard_form.str.contains('식혜'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('식혜'), 'dialect_form'].apply(lambda x: x.replace('식혜', '식게'))
merged_df.loc[merged_df.standard_form.str.contains('아기무덤'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('아기무덤'), 'dialect_form'].apply(lambda x: x.replace('아기무덤', '애기산'))
merged_df.loc[merged_df.standard_form.str.contains('입술'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('입술'), 'dialect_form'].apply(lambda x: x.replace('입술', '입바위'))
merged_df.loc[merged_df.standard_form.str.contains('여드름'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('여드름'), 'dialect_form'].apply(lambda x: x.replace('여드름', '놋싸움'))
merged_df.loc[merged_df.standard_form.str.contains('종아리'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('종아리'), 'dialect_form'].apply(lambda x: x.replace('종아리', '종애'))
merged_df.loc[merged_df.standard_form.str.contains('진달래'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('진달래'), 'dialect_form'].apply(lambda x: x.replace('진달래', '전기꼿'))
merged_df.loc[merged_df.standard_form.str.contains('참기름'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('참기름'), 'dialect_form'].apply(lambda x: x.replace('참기름', '촘지름'))
merged_df.loc[merged_df.standard_form.str.contains('호주머니'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('호주머니'), 'dialect_form'].apply(lambda x: x.replace('호주머니', '게와'))

## 4) 신체
merged_df.loc[merged_df.standard_form.str.contains('겨드랑이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('겨드랑이'), 'dialect_form'].apply(lambda x: x.replace('겨드랑이', '저깽이'))
merged_df.loc[merged_df.standard_form.str.contains('겨드랑이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('겨드랑이'), 'dialect_form'].apply(lambda x: x.replace('저껭이', '저깽이'))
merged_df.loc[merged_df.standard_form.str.contains('겨드랑이'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('겨드랑이'), 'dialect_form'].apply(lambda x: x.replace('조깽이', '저깽이'))
merged_df.loc[merged_df.standard_form.str.contains('어깨'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('어깨'), 'dialect_form'].apply(lambda x: x.replace('어깨', '둑지'))
merged_df.loc[merged_df.standard_form.str.contains('어깨'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('어깨'), 'dialect_form'].apply(lambda x: x.replace('우뚝지', '둑지'))
merged_df.loc[merged_df.standard_form.str.contains('정수리'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('정수리'), 'dialect_form'].apply(lambda x: x.replace('정수리', '상감메'))

## 5) 농사
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('메밀', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('모밀', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('모말', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('모멀', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('모믈', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('마말', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('마멀', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('마물', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('메밀'), 'dialect_form'].apply(lambda x: x.replace('마믈', '모물'))
merged_df.loc[merged_df.standard_form.str.contains('참외'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('참외'), 'dialect_form'].apply(lambda x: x.replace('참외', '참웨'))
merged_df.loc[merged_df.standard_form.str.contains('참외'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('참외'), 'dialect_form'].apply(lambda x: x.replace('촘외', '참웨'))
merged_df.loc[merged_df.standard_form.str.contains('갈퀴'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('갈퀴'), 'dialect_form'].apply(lambda x: x.replace('글갱이', '글겡'))
merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'].apply(lambda x: x.replace('맷돌', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'].apply(lambda x: x.replace('멧돌', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'].apply(lambda x: x.replace('고래', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'].apply(lambda x: x.replace('고레', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('맷돌'), 'dialect_form'].apply(lambda x: x.replace('가래', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'].apply(lambda x: x.replace('멧돌', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'].apply(lambda x: x.replace('맷돌', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'].apply(lambda x: x.replace('고래', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'].apply(lambda x: x.replace('고레', '가레'))
merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('멧돌'), 'dialect_form'].apply(lambda x: x.replace('가래', '가레'))

## 6) 식물
merged_df.loc[merged_df.standard_form.str.contains('감나무'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('감나무'), 'dialect_form'].apply(lambda x: x.replace('감나무', '감낭'))
merged_df.loc[merged_df.standard_form.str.contains('동백나무'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('동백나무'), 'dialect_form'].apply(lambda x: x.replace('동백', '돔박'))
merged_df.loc[merged_df.standard_form.str.contains('동백나무'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('동백나무'), 'dialect_form'].apply(lambda x: x.replace('나무', '낭'))
merged_df.loc[merged_df.standard_form.str.contains('미역'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('미역'), 'dialect_form'].apply(lambda x: x.replace('미역', '메역'))
merged_df.loc[merged_df.standard_form.str.contains('미역'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('미역'), 'dialect_form'].apply(lambda x: x.replace('매역', '메역'))
merged_df.loc[merged_df.standard_form.str.contains('배나무'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('배나무'), 'dialect_form'].apply(lambda x: x.replace('배나무', '배낭'))
merged_df.loc[merged_df.standard_form.str.contains('벚나무'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('벚나무'), 'dialect_form'].apply(lambda x: x.replace('벚나무', '사오기'))
merged_df.loc[merged_df.standard_form.str.contains('소나무'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('소나무'), 'dialect_form'].apply(lambda x: x.replace('소나무', '소낭'))
merged_df.loc[merged_df.standard_form.str.contains('소나무'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('소나무'), 'dialect_form'].apply(lambda x: x.replace('소남', '소낭'))
merged_df.loc[merged_df.standard_form.str.contains('억새'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('억새'), 'dialect_form'].apply(lambda x: x.replace('억새', '어욱'))
merged_df.loc[merged_df.standard_form.str.contains('억새'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('억새'), 'dialect_form'].apply(lambda x: x.replace('어웃', '어욱'))
merged_df.loc[merged_df.standard_form.str.contains('진달래'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('진달래'), 'dialect_form'].apply(lambda x: x.replace('진달래', '전기꼿'))
merged_df.loc[merged_df.standard_form.str.contains('진달래'), 'dialect_form'] = merged_df.loc[merged_df.standard_form.str.contains('진달래'), 'dialect_form'].apply(lambda x: x.replace('창꽃', '전기꼿'))

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import hamming
import numpy as np

## 표준어, 제주어끼리 유사도 분석
## 자카드 유사도와 선형 유사도 비교해서 0.9 이상인 행들 제거
def calculate_similarities(df):
    vectorizer = TfidfVectorizer()

    # 벡터화
    standard_vectors = vectorizer.fit_transform(df['standard_form'])
    dialect_vectors = vectorizer.transform(df['dialect_form'])

    # linear_kernel 유사도
    linear_sim = [linear_kernel(a, b)[0][0] for a, b in zip(standard_vectors, dialect_vectors)]
    df['linear_kernel'] = linear_sim

    # Jaccard 유사도
    jaccard_sim = []
    for a, b in zip(df['standard_form'], df['dialect_form']):
        set_a = set(a.split())
        set_b = set(b.split())
        if not set_a or not set_b:
            jaccard_sim.append(0)
        else:
            jaccard_sim.append(len(set_a & set_b) / len(set_a | set_b))
    df['jaccard_similarity'] = jaccard_sim

    return df

merged_df = calculate_similarities(merged_df)

In [ ]:
merged_df = merged_df.loc[(merged_df.linear_kernel < 0.9) & (merged_df.jaccard_similarity < 0.9)]

In [ ]:
merged_df.shape

(832466, 18)

## JEJU 데이터 전처리

In [ ]:
## 제주어 - 표준어 서로의 답이 있는 경우만 남기기
jeju = jeju.loc[jeju.standard_form != jeju.dialect_form]

## 중복 행 제거
jeju = jeju.drop_duplicates(subset='standard_form', keep='first')
jeju = jeju.drop_duplicates(subset='dialect_form', keep='first')

## 한글, 공백만 남기기
jeju = jeju[jeju['standard_form'].str.contains('^[가-힣\s]*$', regex=True)]
jeju = jeju[jeju['dialect_form'].str.contains('^[가-힣\s]*$', regex=True)]

## 발화 길이 조정
jeju['standard_len'] = jeju.standard_form.apply(lambda x : len(x))
jeju['dialect_len'] = jeju.dialect_form.apply(lambda x : len(x))
jeju = jeju.loc[(jeju.standard_len >= 4) & (jeju.standard_len < 64)]
jeju = jeju.loc[(jeju.dialect_len >= 4) & (jeju.dialect_len < 64)]

## 두 column의 len 차이가 8 보다 작은 데이터만 남기기 --> 8 이상인 경우에 옳은 데이터도 있지만 ()나 이상한 단어가 많이 껴 있는 경우가 많았음
jeju = jeju[abs(jeju.standard_len - jeju.dialect_len) < 8]

## 발화한 사람 기준 제주 거주 기간이 50년 이상인 사람의 발화만 남기기 --> period column의 중위수가 50이라
jeju = jeju.loc[jeju.period >= 50]

In [ ]:
## jeju 데이터는 약 70,000 --> 약 14,000
jeju.shape

(13786, 12)

## life 데이터 전처리 할거 없음

In [ ]:
life.shape

(460, 3)

## je_ko 데이터 전처리

In [ ]:
## 문장 부호 앞에 공백이 있는 경우에 공백 제거
je_ko['standard_form'] = je_ko['standard_form'].apply(lambda x: re.sub(r'\s+([?!.,])', r'\1', x))
je_ko['dialect_form'] = je_ko['dialect_form'].apply(lambda x: re.sub(r'\s+([?!.,])', r'\1', x))

## 중복 행 제거
je_ko = je_ko.drop_duplicates(subset='standard_form', keep='first')
je_ko = je_ko.drop_duplicates(subset='dialect_form', keep='first')

## 불용어 제거
je_ko['dialect_form'] = je_ko.dialect_form.apply(lambda x : x.replace('', ''))
je_ko['standard_form'] = je_ko.standard_form.apply(lambda x : x.replace('', ''))

## 발화길이 조정
je_ko['standard_len'] = je_ko['standard_form'].apply(lambda x : len(x))
je_ko['dialect_len'] = je_ko['dialect_form'].apply(lambda x : len(x))
je_ko = je_ko.loc[(je_ko.standard_len >= 4) & (je_ko.standard_len <= 64) & (je_ko.dialect_len >= 4) & (je_ko.dialect_len <= 64)]

In [ ]:
## 표준어, 제주어끼리 유사도 분석
## 자카드 유사도와 선형 유사도 비교해서 0.9 이상인 행들 제거
def calculate_similarities(df):
    vectorizer = TfidfVectorizer()

    # 벡터화
    standard_vectors = vectorizer.fit_transform(df['standard_form'])
    dialect_vectors = vectorizer.transform(df['dialect_form'])

    # linear_kernel 유사도
    linear_sim = [linear_kernel(a, b)[0][0] for a, b in zip(standard_vectors, dialect_vectors)]
    df['linear_kernel'] = linear_sim

    # Jaccard 유사도
    jaccard_sim = []
    for a, b in zip(df['standard_form'], df['dialect_form']):
        set_a = set(a.split())
        set_b = set(b.split())
        if not set_a or not set_b:
            jaccard_sim.append(0)
        else:
            jaccard_sim.append(len(set_a & set_b) / len(set_a | set_b))
    df['jaccard_similarity'] = jaccard_sim

    return df

je_ko = calculate_similarities(je_ko)

In [ ]:
je_ko = je_ko.loc[(je_ko.linear_kernel < 0.8) & (je_ko.jaccard_similarity < 0.8)]

In [ ]:
df = merged_df.merge(jeju[['standard_form', 'dialect_form']], on = ['standard_form', 'dialect_form'], how = 'outer')\
        .merge(life[['standard_form', 'dialect_form']], on = ['standard_form', 'dialect_form'], how = 'outer')\
        .merge(je_ko[['standard_form', 'dialect_form']], on = ['standard_form', 'dialect_form'], how = 'outer')

In [ ]:
df

/usr/local/lib/python3.10/dist-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/usr/local/lib/python3.10/dist-packages/google/colab/_interactive_table_hint_button.py:178: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_html=dataframe._repr_html_(),  # pylint: disable=protected-access


,category,year,topic,speaker_id,age,sex,occupation,birthplace,education,form,standard_form,dialect_form,isDialect,extracted,standard_len,dialect_len,linear_kernel,jaccard_similarity
0,제주방언 > 사적 대화 > 일상 대화,2020.0,가족,1,50대,여성,전문가 및 관련 종사자,제주,대졸,어 그거 (튀겨그네)/(튀겨서),어 그거 튀겨서,어 그거 튀겨그네,True,(튀겨그네)/(튀겨서),8.0,9.0,0.383816,0.500000
1,제주방언 > 사적 대화 > 일상 대화,2020.0,가족,1,50대,여성,전문가 및 관련 종사자,제주,대졸,돌멩이 해 가지고 실명된사람 (이서)/(있어),돌멩이 해 가지고 실명된사람 있어,돌멩이 해 가지고 실명된사람 이서,True,(이서)/(있어),18.0,18.0,0.852266,0.666667
2,제주방언 > 사적 대화 > 일상 대화,2020.0,가족,1,50대,여성,전문가 및 관련 종사자,제주,대졸,(거난)/(그러니까) 제주도는 워낙 또 돌멩이가 (많아그네)/(많아서),그러니까 제주도는 워낙 또 돌멩이가 많아서,거난 제주도는 워낙 또 돌멩이가 많아그네,True,"(거난)/(그러니까), (많아그네)/(많아서)",23.0,22.0,0.770262,0.500000
3,제주방언 > 사적 대화 > 일상 대화,2020.0,가족,1,50대,여성,전문가 및 관련 종사자,제주,대졸,(겅하니깐)/(그러니깐),그러니깐,겅하니깐,True,(겅하니깐)/(그러니깐),4.0,4.0,0.000000,0.000000
4,제주방언 > 사적 대화 > 일상 대화,2020.0,가족,1,50대,여성,전문가 및 관련 종사자,제주,대졸,데리고 가고 (싶어그네)/(싶어서) 할아버지가(이)/(#이),데리고 가고 싶어서 할아버지가이,데리고 가고 싶어그네 할아버지가이,True,"(싶어그네)/(싶어서), (이)/(#이)",17.0,18.0,0.886922,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"떡이 부서졌어요. 부서지면 부스러기 막 있잖아요, 그 부스러긴 뭐라고 말해?","떡이 부서젼예. 부서지민 부스러기 막 이십니께, 그 부스러긴 뭐엔 갈아?",NaN,NaN,NaN,NaN,NaN,NaN
942288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"누가 아니? 그러니까 다섯 상제, 아들 다섯 개면 다섯 개 다 했었어, 옛날.","누게 아느냐? 게난 거 다섯 상제, 아덜 다섯 개민 다섯 개 다 헤낫어 옛날",NaN,NaN,NaN,NaN,NaN,NaN
942289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,이렇게 열어서 막 죽젓개로 그냥 거 저으려고 하면 얼마나 힘들까봐.,영 열아근에 막 남죽으로 그냥 거 젓젠 허민 얼마나 힘들카부덴.,NaN,NaN,NaN,NaN,NaN,NaN
942290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,밭 갈 때 밭 갈러 가려고 하면.,밧 갈 때 밧 갈레 가젠 하면은.,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('0523_942292row.csv', index = False)